# Setup

In [14]:
# Extract and build the benchmark
!make -C io

!mkdir -p /data/lab1
!io/io-static -c lab1/iofile

cc -Wall -o io-static -DPROGNAME=\"io-static\" io.c -static
cc -Wall -o io-dynamic -DPROGNAME=\"io-dynamic\" io.c -dynamic
Benchmark configuration:
  buffersize: 16384
  totalsize: 16777216
  blockcount: 1024
  operation: create
  path: lab1/iofile
  time: 2.453858376
speed: 6676.83 KBytes/sec


# Reads

## Constant IO/Varying Buffer (with cache, static/dynamic)

* Total size: 16MB (default)
* Buffer: varies from 512 to 512 * 2^16
* (io.c modified to output kbytes/sec in same format as other params)

TODO:
* Run multiple times then average
* Prevent warmup effect
* DTrace and consider probe effect
* Plot

In [13]:
# Setup
BUF_START = 512
BUFFER_SIZES = [512 * 2 ** exp for exp in range(0, 16)]
FILES = ["lab1/iofile", "/dev/zero"]
IO_SIZE = BUFFER_SIZES[-1] # Keep constant at default 16MB (last value of BUFFER_SIZES)

def strs_to_tup(strs):
    if len(strs) == 1:
        return (strs[0].strip(),None)
    else:
        return (strs[0].strip(),strs[1].strip())

def parse(cmd_out):
    tups = map(strs_to_tup, [string.split(":") for string in cmd_out])
    return dict(tups)

# The meat
print("######## Start io-static ########")

static_outputs = []
for filename in FILES:
    for buffer_size in BUFFER_SIZES:
        print("\t ==> Reading from {}".format(filename))
        out = !io/io-static -v -r -b {str(buffer_size)} -t {str(IO_SIZE)} {filename}
        parsed = parse(out)
        static_outputs.append(parsed)
        print("{} bytes {} bytes ({}): {}s".format(parsed['buffersize'], parsed['totalsize'], parsed['speed'], parsed['time']))

print("######## End io-static ########")

print("######## Start io-dynamic ########")

dynamic_outputs = []
for filename in FILES:
    for buffer_size in BUFFER_SIZES:
        print("\t ==> Reading from {}".format(filename))
        out = !io/io-dynamic -v -r -b {str(buffer_size)} -t {str(IO_SIZE)} {filename}
        parsed = parse(out)
        dynamic_outputs.append(parsed)
        print("{} bytes {} bytes ({}): {}s".format(parsed['buffersize'], parsed['totalsize'], parsed['speed'], parsed['time']))

print("######## End io-dynamic ########")

######## Start io-static ########
	 ==> Reading from lab1/iofile


KeyError: 'speed'